In [6]:
import math
import torch
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from IG_Interpreter import Interpreter
from LightGCN import LightGCN
from RecommenderTrainer import RecommenderTrainer
from DataProcessor import DataProcessor
import d2l.torch as d2l

In [ ]:
data_path = "../hw1/ml-1m/ratings.dat"
movies_path = "../hw1/ml-1m/movies.dat"
users_path = "../hw1/ml-1m/users.dat"

# Load and preprocess data
processor = DataProcessor(data_path, movies_path, users_path)
processor.load_data()

/Users/malangtian/Documents/StudyMaterials/Courses/stat992_s25/hw2/DataProcessor.py:145: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


In [ ]:
EMBEDDING_DIM = 64
NUM_LAYERS = 3
LEARNING_RATE = 0.01

# Define and train LightGCN model
model = LightGCN(
    num_users=processor.num_users,
    num_items=processor.num_items,
    embedding_dim=EMBEDDING_DIM,
    n_layers=NUM_LAYERS,
    user_feat_dim=processor.user_features.shape[1],
    item_feat_dim=processor.item_features.shape[1]
)

trainer = RecommenderTrainer(model, 
                             processor, 
                             learning_rate=LEARNING_RATE)
trainer.train()


Epoch 1/5, Loss: -6746522.5000


In [ ]:
interpreter = Interpreter(trainer, processor)
print("Top recommendations for user 0:")
top_movies = interpreter.find_top_recommendations(user_index=0, k=5)